# Expressions (10 min)

In the previous sub-module, {doc}`mod4_2_cell_statistics_band_math`, we executed some simple mathmatical operations on Images; however, sometimes we have multi-step operations or more complicated expressions we wish to apply.

For that, we can use the `.expression()` method built-in to the Image class.

In this tutorial we'll shift back to DMSP-OLS data and take a sneak peak at a process for intercalibrating DMSP data. As noted in {doc}`mod1_2_introduction_to_nighttime_light_data`, this is necessary when analyzing a DMSP-OLS time series. We get into this in much more detail in {doc}`mod3_2_DMSP-OLS_intercalibration`, but for now, we'll introduce the use of expressions in Google Earth Engine.

**Prerequisites:**
- Make sure you have Python, Jupyter notebooks, and `ee` and `geemap` installed and are familiar with these packages
- If not, you'll want to review: 
    - {doc}`mod2_1_getting_started_with_Python`
    - {doc}`mod2_2_introduction_to_Jupyter_notebooks`
    - {doc}`mod2_3_introduction_to_GEE`
- You may also want to try this exercise to get familiar with GEE and the Python `geemap` and `ee` libraries:
    - {doc}`mod2_4_practical_exercise-image_visualization`

**Our tasks in this exercise:**
1. Invert a 1996 DMSP-OLS composite using built-in functions.
2. Conduct an equivalent operation using `.expression()`
3. Apply a simple polynomial formula for DMSP-OLS intercalibration co-efficients to a 1996 composite.

## Invert an image with Image functions


### Initialize map with DMSP-OLS layer
First, as we've often done, let's initialize a map object.

Here we'll center our scene on Mexico City.

And we'll pull the annual DMSP-OLS nightime lights composite for 1996, using the "stable lights" band.

In [11]:
# import geemap and ee for our Python session
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# # initialize our map and center on Mexico City, Mexico
lat = 19.43
lon = -99.13
dmspMap = geemap.Map(center=[lat,lon],zoom=6)
dmspMap.add_basemap('SATELLITE')

# get 1996 composite, apply mask, and add as layer
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F121996").select('stable_lights')
dmspMap.addLayer(dmsp1996.mask(dmsp1996), {}, "DMSP-OLS 1996", opacity=0.75)

In [2]:
dmspMap

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

### Invert an image

Let's say we want to invert an image. We know our max value (our Digital Number, if you recall) for DMSP-OLS images is 63. So if we multiply everything by -1 and then add 63. So, for example what was a DN of 63 before (the max) is now a 0 and vice versa.

We can do this using the built-in functions `.multiply` and `.add()`

**Note**: we selected our band, `stable_lights` above and saved to our variable `dmsp1996`, but if we didnt before we would have to do that here to explicitly apply our operation, including expression, to a specific band.

In [3]:
dmsp1996_inv = dmsp1996.multiply(-1).add(63)

### Now lets create a slider map but adjust our min/max values.

We'll compare our original image with the transformed (inverted) image we just created.

**Note:** to ensure we're comparing them on the same scale, we're using the visual parameters in the `.addLayer()` function, which allows us to do a few things to our image, such as clip the image to "min" and "max" values.

In [10]:
invMap = geemap.Map(center=[lat,lon],zoom=6)
invMap.add_basemap('SATELLITE')

left_layer = geemap.ee_tile_layer(dmsp1996, {'min':0,'max':63}, 'DMSP NTL 1996', opacity=0.75)
right_layer = geemap.ee_tile_layer(dmsp1996_inv, {'min':0,'max':63}, 'DMSP NTL 1996 inverse', opacity=0.75)

invMap.split_map(left_layer=left_layer, right_layer=right_layer)
invMap

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

## Invert image with `.expression()`

Now, we'll perform the same operation, but using the `expression()` method.

The `ee.Image.expression() ` method takes a string input as the formula. The second argument is a dictionary with key-value pairs, where the keys are the characters in our string we want to use as variables, (e.g. "X") and the values are the corresponding data -- a particular Image band in this case.

In [6]:
inv_formula = "(X*-1) + 63"

# we plug this formula in, identify our variable "X" and set it to our 1996 DMSP-OLS "stable_lights" band
dmsp1996_inv2 = dmsp1996.expression(inv_formula, {'X':dmsp1996})

### Gut-check visualization...
If we inspected `dmsp1996_inv` and `dmsp1996_inv2` analytically we'd see they were identical. For now, you can see visually that they are the same.

In [12]:
invMap2 = geemap.Map(center=[lat,lon],zoom=6)
invMap2.add_basemap('SATELLITE')

left_layer = geemap.ee_tile_layer(dmsp1996_inv, {'min':0,'max':63}, 'DMSP NTL 1996 inv method1', opacity=0.75)
right_layer = geemap.ee_tile_layer(dmsp1996_inv2, {'min':0,'max':63}, 'DMSP NTL 1996 inverse method2', opacity=0.75)

invMap2.split_map(left_layer=left_layer, right_layer=right_layer)
invMap2

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

It's equivalent!

So...why use the `expression()` method instead of the built-in functions?

This method used a couple very simple operations, but it can be necessary to use more complex formulae. 

It can be easier to read an expression written in a form (like a string) that we're familiar with. It may also be easier to dynamically update formulae with different variables when using the `.expression()` approach. We'll show a practical use of this in {doc}`mod3_2_DMSP-OLS_intercalibration`.


### Apply a polynomial function to calibrate a DMPS-OLS image
We cover DMSP-OLS intercalibration in more detail in {doc}`mod3_2_DMSP-OLS_intercalibration`, but as an illustrative example of expressions, we're going to look at this intercalibration formula, which  applies a series of coefficients to an input DMSP-OLS image to get an "adjusted" image that corrects for sensor variation (technical paper here {cite}`elvidge2009fifteen`):

These coefficients map to the formula:
$X' = C_{0} + C_{1}*X + C_{2}*X^{2}$

Where:
- X: the input image, represented as a 2-dimensional matrix (recall these images are panchromatic so there is only one channel of light)
- $C_{0}, C_{1}, C_{2}$: the calibration coefficients that are assigned to each satellite
- X': the calibrated image

This is a table of the coefficients created using this method corresponding to specific DMSP-OLS satellite-year data:

```{figure} img/mod2-2-intercalib_coef.png
---
name: intercalib_coefficients
---
DMSP-OLS intercalibration {cite}`jiang2017assessing`
```

For 1996, there is only one satellite, F12, so we can reference the appropriate coefficients for F121996 from our table above:

- $C_{0}$ = -0.0959
- $C_{1}$ = 1.2727
- $C_{2}$ = -0.0040


We add our coefficients to the appropriate terms of the polynomial and set our input image as the X variable and save this formula as a string.

In [8]:
# set our formula
F121996cal = '-0.0959 + (1.2727 * X) + (-0.0040 * X * X)'

# apply our expression to our 1996 composite
dmsp1996_clbr = dmsp1996.expression(F121996cal,{'X':dmsp1996})

### Visualize

In [13]:
calMap = geemap.Map(center=[lat,lon],zoom=6)
calMap.add_basemap('SATELLITE')

left_layer = geemap.ee_tile_layer(dmsp1996.mask(dmsp1996), {'min':0,'max':63}, 'DMSP NTL 1996', opacity=0.75)
right_layer = geemap.ee_tile_layer(dmsp1996_clbr.mask(dmsp1996_clbr), {'min':0,'max':63}, 'DMSP NTL 1996 adjusted', opacity=0.75)

calMap.split_map(left_layer=left_layer, right_layer=right_layer)
calMap

Map(center=[19.43, -99.13], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

### A subtle adjustment...

Note that visually, the changes are hard to detect, but if you zoom in, you can see that the adjusted image has brighter values around the edges of the urban areas.

If we were actually conducting inter-calibration, we'd also clip the adjusted image to specific minimum and max values to account for the fact that some DN values are above our 63 max (you guessed it, more on that in {doc}`mod3_2_DMSP-OLS_intercalibration`).

But for now, you know how use `.expression()` to perform more complex operations on Images!

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```